### 1. 단어 표현방법
* 국소표현(Local Representation) : 단어 그차제만 고려, 이산 표현
* 분산표현(Distributed Representation) : 표현하고자 하는 단어 주변까지 고려하는 방법

### 2. Bag of Words(BoW)

* 단어의 등장순서를 고려하지 않고 빈도수 기반의 단어 표현방법
    * 각 단어에 고유한 정수 인덱스 부여
    * 각 인덱스 위치에 단어 토큰의 등장횟수를 기록한 벡터 생성

In [1]:
from konlpy.tag import Okt
okt = Okt()

In [2]:
def build_bag_of_words(document):
    document = document.replace('.', '')
    tokenized_document = okt.morphs(document)
    
    word_to_index = {}
    bow = []
    
    for word in tokenized_document:
        if word not in word_to_index.keys():
            word_to_index[word] = len(word_to_index)
            bow.insert(len(word_to_index) - 1, 1) # Bow에 전부 기본값 1을 넣음
        else:
            # 재등장하는 단어의 인덱스
            index = word_to_index.get(word)
            # 재등장하는 단어는 해당 인텍스의 위치에 1을 더한다
            bow[index] += 1
    return word_to_index, bow

In [3]:
doc1 = "정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다."
vocab, bow = build_bag_of_words(doc1)
print('vocabulary:', vocab)
print('bag of words vector:', bow)

vocabulary: {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9}
bag of words vector: [1, 2, 1, 1, 2, 1, 1, 1, 1, 1]


In [4]:
doc2 = '소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.'
vocab, bow = build_bag_of_words(doc2)
print('vocabulary:', vocab)
print('bag of words vector:', bow)

vocabulary: {'소비자': 0, '는': 1, '주로': 2, '소비': 3, '하는': 4, '상품': 5, '을': 6, '기준': 7, '으로': 8, '물가상승률': 9, '느낀다': 10}
bag of words vector: [1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1]


In [5]:
doc3 = doc1 + doc2
vocab, bow = build_bag_of_words(doc3)
print('vocabulary:', vocab)
print('bag of words vector:', bow)

vocabulary: {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9, '는': 10, '주로': 11, '소비': 12, '상품': 13, '을': 14, '기준': 15, '으로': 16, '느낀다': 17}
bag of words vector: [1, 2, 1, 2, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1]


In [6]:
doc3

'정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다.소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.'

### 3. CounterVectorizer 클래스로 Bow 만들기

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
corpus = ['you know I want your love. because I love you.']
vector = CountVectorizer()

print('bag of words vector:', vector.fit_transform(corpus).toarray())
print('vocabualry:', vector.vocabulary_)

bag of words vector: [[1 1 2 1 2 1]]
vocabualry: {'you': 4, 'know': 1, 'want': 3, 'your': 5, 'love': 2, 'because': 0}


In [9]:
print('bag of words vector:', vector.fit_transform([doc1]).toarray())
print('vocabualry:', vector.vocabulary_)  # 단순 띄어쓰기로만 구분, 의무구분 없어 성능 떨어짐

bag of words vector: [[1 1 1 1 1 1 1]]
vocabualry: {'정부가': 6, '발표하는': 4, '물가상승률과': 2, '소비자가': 5, '느끼는': 0, '물가상승률은': 3, '다르다': 1}


* Note : CounterVectorizer는 띄어쓰기 기준으로 단어를 토큰화하기 때문에 한국어와 같은 경우는 제대로 만들어지지 않음

### 4. 불용어를 제거한 Bow 만들기

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

In [11]:
text = ["Family is not an important thing. It's everything."]
stop_words = ['the', 'a', 'an', 'is', 'not']
vect = CountVectorizer(stop_words=stop_words)

print('bag of words vector:', vect.fit_transform(text).toarray())
print('vocabulary:', vect.vocabulary_)

bag of words vector: [[1 1 1 1 1]]
vocabulary: {'family': 1, 'important': 2, 'thing': 4, 'it': 3, 'everything': 0}


In [12]:
## CounterVectorizer에서 제공하는 불용어 사용
text = ["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words='english')
print('bag of words vector:', vect.fit_transform(text).toarray())
print('vocabulary:', vect.vocabulary_)

bag of words vector: [[1 1 1]]
vocabulary: {'family': 0, 'important': 1, 'thing': 2}


In [13]:
### NLTK에서 지원하는 불용어 사용
text = ["Family is not an important thing. It's everything."]
stop_words = stopwords.words('english')
vect = CountVectorizer(stop_words=stop_words)
print('bag of words vector:', vect.fit_transform(text).toarray())
print('vocabulary:', vect.vocabulary_)

bag of words vector: [[1 1 1 1]]
vocabulary: {'family': 1, 'important': 2, 'thing': 3, 'everything': 0}


### 5. 문서 단어 행렬(Document-Term Matrix, DTM)
* 서로 다른 문서들의 BoW들을 결합한 표현방법인 문서단어 행렬 방법
* 다수의 문서에서 등장하는 각 단어들의 빈도를 행렬로 표현한 것
* 즉, 각 문서에 대한 BoW를 하나의 행렬로 만든 것임
---
* 한계 : sparse matrix 문제 발생, 불용어 제거가 중요  ==> 중요한 단어에 가중치 부여하는 TF-IDF 사용

### 6. TF-IDF(Term Frequency-Inverse Document Frequency)

* 각 단어에 대한 중요도를 계산할 수 있는 TF-IDF 가중치
* 단어의 빈도와 역문서 빈도를 사용하여 DTM 내의 각 단어들마다 중요한 정도를 가중치로 설정
* TF-IDF는 주로 **문서의 유사도를 구하는 작업**, **검색 시스템에서 검색 결과의 중요도를 정하는 작업**, **문서 내에서 특정 단어의 중요도를 구하는 작업** 등에 쓰임
---
* tf(d, t) : 특정 문서 d에서 특정단어 t의 등장 횟수
* df(t) : 특정 단어 t가 등장한 문서의 수
* idf(d, t) : df(t)에 반비례하는 수.

$$idf(d, t) = log(\frac{n}{1+df(t)})$$

In [14]:
from math import log

docs = [
  '먹고 싶은 사과',
  '먹고 싶은 바나나',
  '길고 노란 바나나 바나나',
  '저는 과일이 좋아요'
] 

vocab = list(set(w for doc in docs for w in doc.split()))
print(vocab)
vocab.sort()
print(vocab)

['싶은', '길고', '좋아요', '노란', '과일이', '저는', '바나나', '먹고', '사과']
['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']


In [15]:
# 총 문서수
N = len(docs)

def tf(t, d):
    return d.count(t)

def idf(t):
    df = 0
    for doc in docs:
        df += t in doc  # 문서에 존재하면 1씩 더하기
    return log(N / (1 + df))

def tfidf(t, d):
    return tf(t, d) * idf(t)

In [16]:
# tf 구하기
result = []

for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j] # 단어 하나씩 설정
        result[-1].append(tf(t,d))

In [17]:
result

[[0, 0, 0, 1, 0, 1, 1, 0, 0],
 [0, 0, 0, 1, 1, 0, 1, 0, 0],
 [0, 1, 1, 0, 2, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 1, 1]]

In [18]:
import pandas as pd

tf_ = pd.DataFrame(result, columns = vocab)

In [19]:
tf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


In [21]:
## IDF 구하기
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))
result

[0.6931471805599453,
 0.6931471805599453,
 0.6931471805599453,
 0.28768207245178085,
 0.28768207245178085,
 0.6931471805599453,
 0.28768207245178085,
 0.6931471805599453,
 0.6931471805599453]

In [24]:
idf_ = pd.DataFrame(result, index=vocab, columns=['IDF'])
idf_

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


In [25]:
result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        result[-1].append(tfidf(t, d))

In [29]:
result

[[0.0,
  0.0,
  0.0,
  0.28768207245178085,
  0.0,
  0.6931471805599453,
  0.28768207245178085,
  0.0,
  0.0],
 [0.0,
  0.0,
  0.0,
  0.28768207245178085,
  0.28768207245178085,
  0.0,
  0.28768207245178085,
  0.0,
  0.0],
 [0.0,
  0.6931471805599453,
  0.6931471805599453,
  0.0,
  0.5753641449035617,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.6931471805599453,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.6931471805599453,
  0.6931471805599453]]

In [30]:
tfidf_ = pd.DataFrame(result, columns=vocab)
tfidf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


### 7. 사이킷런을 이용한 DTM과 TF-IDF 실습

* CountVectorizer를 사용하면 DTM을 만들 수 있음

In [31]:
from sklearn.feature_extraction.text import CountVectorizer

In [32]:
corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]

In [33]:
## DTM 만들기
vector = CountVectorizer()
print(vector.fit_transform(corpus).toarray())

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]


In [34]:
print(vector.vocabulary_)

{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]

In [36]:
tfidf = TfidfVectorizer().fit(corpus)
print(tfidf.transform(corpus).toarray())
print(tfidf.vocabulary_)

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}
